<a href="https://colab.research.google.com/github/cicl-iscl/SemEval_2025_Task1_Jiaong_Ruitong_Yue/blob/main/goldValwithPlainbertResult.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

def calculate_scores_per_compound(train_file, submission_file):
    # Load the files as DataFrames
    train_df = pd.read_csv(train_file, sep='\t')
    submission_df = pd.read_csv(submission_file, sep='\t')

    # Ensure the expected_order is in list format
    train_df['expected_order'] = train_df['expected_order'].apply(eval)
    submission_df['expected_order'] = submission_df['expected_order'].apply(eval)

    # Create a mapping of compound to expected_order indices in the training file
    train_indices = {
        row['compound']: {image: idx for idx, image in enumerate(row['expected_order'])}
        for _, row in train_df.iterrows()
    }

    # Initialize a dictionary to store scores per compound
    scores = {}

    # Compare the expected_order in the submission file
    for _, row in submission_df.iterrows():
        compound = row['compound']
        if compound in train_indices:
            train_order = train_indices[compound]
            submission_order = {image: idx for idx, image in enumerate(row['expected_order'])}

            # Initialize score for the compound
            compound_score = 0

            # Calculate the score based on index matches
            for image, idx in submission_order.items():
                if image in train_order:
                    if train_order[image] != idx:
                        compound_score -= 1  # Non-matching index decreases score

            # Store the score for the compound
            scores[compound] = compound_score

    return scores

# Specify file paths
train_file_path = '/content/gdrive/MyDrive/subtask_a_train.tsv'
submission_file_path = '/content/gdrive/MyDrive/submission.tsv'

# Calculate and print the scores per compound
scores = calculate_scores_per_compound(train_file_path, submission_file_path)

# Filter compounds with high negative scores
negative_scores = {compound: score for compound, score in scores.items() if score < 0}
sorted_negative_scores = dict(sorted(negative_scores.items(), key=lambda item: item[1]))

# Print compounds with high negative scores
print("Compounds with high negative scores:")
for compound, score in sorted_negative_scores.items():
    print(f"Compound: {compound}, Score: {score}")